In [1]:
# Fix sys path to root directory so locally defined packages can be found
import sys
sys.path.append('C:\\Users\\2270577A\\PycharmProjects\\PhD_LCA_TEA')


In [2]:
import numpy as np
import pickle

from config import settings


In [3]:
carbon_percentage_data_raw = np.array([60.04, 59.49, 45.7, 64.80, 1.38, 89.85, 42.0, 46.8, 65.29, 79.34, 35, 71, 33.1])
source = "10.1016/j.biortech.2017.06.177"


In [4]:
# function to reject outliers from a numpy array
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [5]:
# show data
carbon_percentage_data_raw

array([60.04, 59.49, 45.7 , 64.8 ,  1.38, 89.85, 42.  , 46.8 , 65.29,
       79.34, 35.  , 71.  , 33.1 ])

In [6]:
# Remove outliers from array and show new array
carbon_percentage_data = reject_outliers(carbon_percentage_data_raw, m=2)
carbon_percentage_data

array([60.04, 59.49, 45.7 , 64.8 , 89.85, 42.  , 46.8 , 65.29, 79.34,
       35.  , 71.  , 33.1 ])

In [7]:
mean_carbon_percentage = np.mean(carbon_percentage_data) / 100
std_carbon_percentage = np.std(carbon_percentage_data) / 100
print("Mean:", mean_carbon_percentage)
print("Std:", std_carbon_percentage)


Mean: 0.5770083333333335
Std: 0.16894895411698235


In [8]:
# Display triangular distribution data on recalcitrant fraction as stored in settings
recalcitrant_carbon_fraction_data = settings.data.biochar.recalcitrant_fraction
recalcitrant_carbon_fraction_data


<Box: {'lower': 0.8, 'mode': 0.9, 'upper': 0.97, 'source': ['https://doi.org/10.1111/gcbb.12266', 'https://doi.org/10.1021/es902266r']}>

In [10]:
output_dict = {"Biochar carbon fraction": {"Mean": mean_carbon_percentage, "Std": std_carbon_percentage},
               "Biochar recalcitrant carbon fraction": {"Lower": recalcitrant_carbon_fraction_data.lower, "Mode": recalcitrant_carbon_fraction_data.mode, "Upper": recalcitrant_carbon_fraction_data.upper}
               }

# Store Performance summary data frame of best performing model
with open(r"results\biochar_properties_results", "wb") as f:
    pickle.dump(output_dict, f)

# Also save to data folder
with open(r"C:\Users\2270577A\PycharmProjects\PhD_LCA_TEA\data\biochar_properties_results", "wb") as f:
    pickle.dump(output_dict, f)
